In [2]:
import os

import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression

In [3]:
CSV_LOCATION = "peace_sys.csv"

In [4]:
csv = pd.read_csv(CSV_LOCATION)

In [ ]:
KEYS = ['ID1.1Over', '']

In [5]:
csv.head(5)

,Name,SCCS,Coder,ID1.1Over,ID1.2Ethno,Int2.1Mar,Int2.2Econ,Int2.3Pol,Int2.4Hist,Dep3.1Sec,...,CM8.5Peace,CM8.6War,Lead9.1P,Lead9.2War,Cult10.1Com,Cult10.2Diff,Comp10.3,InComp10.4,PSys,PSysRec
0,Gilbertese,107,KA,9.0,2.0,3,9.0,9,9,9,...,9,9,9,9,9,9,9,9,2,0
1,Marshallese,108,EC,1.0,9.0,9,1.0,9,9,9,...,9,2,9,4,9,9,9,9,2,0
2,E. Pomo,135,"KA, EC",9.0,9.0,9,9.0,9,9,1,...,9,9,2,9,9,9,9,9,2,0
3,Popoluca,154,KA,9.0,3.0,9,3.0,9,9,9,...,9,9,9,9,9,9,9,9,2,0
4,Konso,35,KA,9.0,2.0,3,4.0,9,4,9,...,9,9,9,9,3,9,3,3,2,0
